In [1]:
import requests
import pandas as pd
import time
import os
from datetime import datetime, UTC
from itertools import product # Para poder realizar todas las combinaciones posibles de los términos
from dotenv import load_dotenv

In [2]:
# Cargar variables de entorno
load_dotenv()
API_URL = os.getenv("ITUNES_API_URL")

In [3]:
# Configuración
CARPETA_DATOS = "../data/data_raw" # Carpeta donde se guardarán los datos
LOG_TERMS = "terminos_usados.txt" # Archivo donde se guardarán los términos usados
os.makedirs(CARPETA_DATOS, exist_ok=True) # Crear la carpeta si no existe

In [4]:
# Generar combinaciones 'aa' a 'zz'
TERMINOS = [''.join(p) for p in product('abcdefghijklmnopqrstuvwxyz', repeat=2)]
TERMINOS_POR_DIA = 97  # 97 términos × 200 filas ≈ 19.400 canciones/día × 7 días = 135.800 filas totales

In [5]:
# Cargar términos ya usados
def cargar_terminos_usados():
    if os.path.exists(LOG_TERMS):
        with open(LOG_TERMS, "r") as f:
            return set(line.strip() for line in f.readlines())
    return set()

In [6]:
# Guardar término como usado
def guardar_termino_usado(term):
    with open(LOG_TERMS, "a") as f:
        f.write(f"{term}\n")

In [7]:
# Buscar en la API de iTunes usando la URL desde .env
def buscar_itunes(term, limit=200):
    params = {
        "term": term,
        "limit": limit,
        "country": "US",
        "media": "music"
    }

    try:
        response = requests.get(API_URL, params=params)
        if response.status_code == 200:
            results = response.json().get("results", [])
            df = pd.json_normalize(results)
            if not df.empty:
                df["checked_at"] = datetime.now(UTC).isoformat()
                return df
    except Exception as e:
        print(f"❌ Error con '{term}': {e}")
    return pd.DataFrame()

In [8]:
# Ejecutar scraping para múltiples términos por día
def ejecutar_scrape_diario():
    usados = cargar_terminos_usados()
    pendientes = [t for t in TERMINOS if t not in usados][:TERMINOS_POR_DIA]

    if not pendientes:
        print("✅ Todos los términos han sido usados.")
        return

    dfs = []

    for termino in pendientes:
        print(f"🔍 Buscando: '{termino}'")
        df = buscar_itunes(termino)
        if not df.empty:
            dfs.append(df)
            guardar_termino_usado(termino)
            print(f"✅ {len(df)} resultados para '{termino}'")
        else:
            print(f"⚠️ Sin resultados para '{termino}'")
        time.sleep(1)

    if dfs:
        df_total = pd.concat(dfs, ignore_index=True)
        hoy = datetime.now().strftime("%Y-%m-%d")
        archivo_salida = f"{"../data/data_raw"}/itunes_{hoy}.csv"
        df_total.to_csv(archivo_salida, index=False)
        print(f"📦 Guardados {len(df_total)} registros en '{archivo_salida}'")

if __name__ == "__main__":
    ejecutar_scrape_diario()

🔍 Buscando: 'de'
⚠️ Sin resultados para 'de'
🔍 Buscando: 'do'
⚠️ Sin resultados para 'do'
🔍 Buscando: 'du'
⚠️ Sin resultados para 'du'
🔍 Buscando: 'ep'
⚠️ Sin resultados para 'ep'
🔍 Buscando: 'eu'
✅ 157 resultados para 'eu'
🔍 Buscando: 'ev'
✅ 200 resultados para 'ev'
🔍 Buscando: 'ew'
✅ 199 resultados para 'ew'
🔍 Buscando: 'ex'
✅ 201 resultados para 'ex'
🔍 Buscando: 'ey'
✅ 200 resultados para 'ey'
🔍 Buscando: 'ez'
✅ 201 resultados para 'ez'
🔍 Buscando: 'fa'
✅ 200 resultados para 'fa'
🔍 Buscando: 'fb'
✅ 98 resultados para 'fb'
🔍 Buscando: 'fc'
✅ 172 resultados para 'fc'
🔍 Buscando: 'fd'
✅ 200 resultados para 'fd'
🔍 Buscando: 'fe'
✅ 200 resultados para 'fe'
🔍 Buscando: 'ff'
✅ 200 resultados para 'ff'
🔍 Buscando: 'fg'
✅ 201 resultados para 'fg'
🔍 Buscando: 'fh'
✅ 202 resultados para 'fh'
🔍 Buscando: 'fi'
✅ 200 resultados para 'fi'
🔍 Buscando: 'fj'
✅ 200 resultados para 'fj'
🔍 Buscando: 'fk'
✅ 200 resultados para 'fk'
🔍 Buscando: 'fl'
✅ 200 resultados para 'fl'
🔍 Buscando: 'fm'
✅ 202 result